In [1]:
import os
import sys
import tensorflow as tf
import numpy as np
import math
import timeit
import scipy

#import matplotlib.pyplot as plt

import provider
import TR
#%matplotlib inline

In [2]:
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

Using device:  /device:GPU:0


In [3]:
class CentralNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2,channel_3,channel_4, num_classes,c1,c2,c3,c4):
        super().__init__()

#         pass
        initializer = tf.variance_scaling_initializer(scale=2.0)
        self.conv1 = tf.layers.Conv2D(channel_1, kernel_size=(3,3), 
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer())
        self.conv2 = tf.layers.Conv2D(channel_1, kernel_size=(3,3), 
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer())
        self.maxpool1 = tf.layers.MaxPooling2D(pool_size = (2,2), 
                               strides = (2,2), padding='SAME')
        
        self.conv3 = tf.layers.Conv2D(channel_2, kernel_size=(3,3), 
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer())        
        self.conv4 = tf.layers.Conv2D(channel_2, kernel_size=(3,3), 
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer())
        self.maxpool2 = tf.layers.MaxPooling2D(pool_size = (2,2), 
                               strides = (2,2), padding='SAME')
        
        self.conv5 = tf.layers.Conv2D(channel_3, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-256
        self.conv6 = tf.layers.Conv2D(channel_3, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-256
        self.conv7 = tf.layers.Conv2D(channel_3, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-256
        self.maxpool3 = tf.layers.MaxPooling2D(pool_size = (2,2), 
                               strides = (2,2), padding='SAME')

        self.conv8 = tf.layers.Conv2D(channel_4, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.conv9 = tf.layers.Conv2D(channel_4, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.conv10 = tf.layers.Conv2D(channel_4, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.maxpool4 = tf.layers.MaxPooling2D(pool_size = (2,2), 
                               strides = (2,2), padding='SAME')

        self.conv11 = tf.layers.Conv2D(channel_4, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.conv12 = tf.layers.Conv2D(channel_4, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.conv13 = tf.layers.Conv2D(channel_4, kernel_size=(3,3),
                                      strides=(1,1),padding="SAME",
                                      activation = tf.nn.relu,use_bias=True,
                                      kernel_initializer = initializer,
                                      bias_initializer=tf.zeros_initializer()) # conv3-512
        self.maxpool5 = tf.layers.MaxPooling2D(pool_size = (2,2), 
                               strides = (2,2), padding='SAME')
                                     
        self.fc4c1 = tf.layers.Dense(c1, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c14c2 = tf.layers.Dense(c2, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4c2 = tf.layers.Dense(c2, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c24c3 = tf.layers.Dense(c3, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4c3 = tf.layers.Dense(c3, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c34c4 = tf.layers.Dense(c4, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.fc4c4 = tf.layers.Dense(c4, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        self.c44c5 = tf.layers.Dense(4096, activation=tf.nn.relu,use_bias=False,
                                  kernel_initializer=initializer,
                                  bias_initializer=tf.zeros_initializer())
        
        self.flatten = tf.keras.layers.Flatten()
        
        self.fc1=tf.layers.Dense(4096, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fc2=tf.layers.Dense(4096, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fc3=tf.layers.Dense(1000, activation = tf.nn.relu, 
                        kernel_initializer=initializer)
        self.fc4=tf.layers.Dense(num_classes, activation = None, 
                        kernel_initializer=initializer)
     
    def call(self, x1,x2, training=None):
        scores = None
        ###

        x1_conv1 = self.conv1(x1)
        x1_conv2 = self.conv2(x1_conv1)
        x1_maxpool1 = self.maxpool1(x1_conv2)
        h1_x1=self.flatten(x1_maxpool1)
        x2_conv1 = self.conv1(x2)
        x2_conv2 = self.conv2(x2_conv1)
        x2_maxpool1 = self.maxpool1(x2_conv2)
        h1_x2=self.flatten(x2_maxpool1)
        hc1=self.fc4c1(h1_x1)+self.fc4c1(h1_x2)
        ##conv3-64
        x1_conv3 = self.conv3(x1_maxpool1)
        x1_conv4 = self.conv4(x1_conv3)
        x1_maxpool2 = self.maxpool2(x1_conv4)
        h2_x1=self.flatten(x1_maxpool2)
        x2_conv3 = self.conv3(x2_maxpool1)
        x2_conv4 = self.conv4(x2_conv3)
        x2_maxpool2 = self.maxpool2(x2_conv4)
        h2_x2=self.flatten(x2_maxpool2)
        hc2=self.fc4c2(h2_x1)+self.fc4c2(h2_x2)+self.c14c2(hc1)
        ##conv3-128
        x1_conv5 = self.conv5(x1_maxpool2)
        x1_conv6 = self.conv6(x1_conv5)
        x1_conv7 = self.conv7(x1_conv6)
        x1_maxpool3 = self.maxpool3(x1_conv7)
        h3_x1=self.flatten(x1_maxpool3)
        x2_conv5 = self.conv5(x2_maxpool2)
        x2_conv6 = self.conv6(x2_conv5)
        x2_conv7= self.conv7(x2_conv6)
        x2_maxpool3= self.maxpool3(x2_conv7)
        h3_x2=self.flatten(x2_maxpool3)
        hc3=self.fc4c3(h3_x1)+self.fc4c3(h3_x2)+self.c24c3(hc2)
        ##conv3-256
        x1_conv8 = self.conv8(x1_maxpool3)
        x1_conv9 = self.conv9(x1_conv8)
        x1_conv10 = self.conv10(x1_conv9)
        x1_maxpool4 = self.maxpool4(x1_conv10)
        h4_x1=self.flatten(x1_maxpool4)
        x2_conv8 = self.conv8(x2_maxpool3)
        x2_conv9 = self.conv9(x2_conv8)
        x2_conv10 = self.conv10(x2_conv9)
        x2_maxpool4= self.maxpool4(x2_conv10)
        h4_x2=self.flatten(x2_maxpool4)
        hc4=self.fc4c4(h4_x1)+self.fc4c4(h4_x2)+self.c34c4(hc3)
        ##conv3-512
        x1_conv11 = self.conv11(x1_maxpool4)
        x1_conv12 = self.conv12(x1_conv11)
        x1_conv13 = self.conv13(x1_conv12)
        x1_maxpool5 = self.maxpool5(x1_conv13)
        h5_x1=self.flatten(x1_maxpool5)
        x2_conv11 = self.conv11(x2_maxpool4)
        x2_conv12 = self.conv12(x2_conv11)
        x2_conv13 = self.conv13(x2_conv12)
        x2_maxpool5= self.maxpool5(x2_conv13)
        h5_x2=self.flatten(x2_maxpool5)
        hc5=self.fc1(h5_x1)+self.fc1(h5_x2)+self.c44c5(hc4)    #x1 and x2 have the same size    #4096
        dense2=self.fc2(hc5) #4096
        dense3=self.fc3(dense2)#1000
        scores= self.fc4(dense3)#num_classes
        return scores

def placeholder_inputs(batch_size, img_rows=66, img_cols=200, points=16384, separately=False):
    imgs_pl = tf.placeholder(tf.float32, shape=(batch_size, img_rows, img_cols, 3))
    pts_pl = tf.placeholder(tf.float32, shape=(batch_size, points, 3))
    if separately:
        speeds_pl = tf.placeholder(tf.float32, shape=(batch_size))
        angles_pl = tf.placeholder(tf.float32, shape=(batch_size))
        labels_pl = [speeds_pl, angles_pl]
    labels_pl = tf.placeholder(tf.float32, shape=(batch_size, 2))
    return imgs_pl, pts_pl, labels_pl


def get_loss(pred, label, l2_weight=0.0001):
    diff = tf.square(tf.subtract(pred, label))
    train_vars = tf.trainable_variables()
    l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in train_vars[1:]]) * l2_weight
    loss = tf.reduce_mean(diff + l2_loss)

    return loss



In [ ]:
tf.reset_default_graph()

channel_1, channel_2,channel_3,channel_4, num_classes,c1,c2,c3,c4=64,128,256,512,10,64,128,256,512
model = CentralNet(channel_1, channel_2,channel_3,channel_4, num_classes,c1,c2,c3,c4)
with tf.device(device):
    x1 = tf.zeros((64, 32, 32,3))
    x2 = tf.zeros((64, 32, 32,3))
    scores = model(x1,x2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    scores_np = sess.run(scores)
    print(scores_np.shape)

In [7]:
BATCH_SIZE=8
MAX_EPOCH=1
learning_rate = 3e-5
channel_1, channel_2,channel_3,channel_4, num_classes,c1,c2,c3,c4=64,128,256,512,2,64,128,256,512

def model_init_fn(inputs1,inputs2, is_training):
    model = None

    model = CentralNet(channel_1, channel_2,channel_3,channel_4, num_classes,c1,c2,c3,c4)

    return model(inputs1,inputs2)

def optimizer_init_fn():
    optimizer = None

    optimizer = tf.train.AdamOptimizer()

    return optimizer

TR.train_part34(model_init_fn, optimizer_init_fn,100,device,BATCH_SIZE)

Starting epoch 0
mean loss: 115723.451701
accuracy (angle): 0.095238
accuracy (speed): 0.161376
eval mean loss: 0.682393
eval accuracy (angle): 0.135638
eval accuracy (speed): 0.223404
Starting epoch 1
mean loss: 0.860238
accuracy (angle): 0.124008
accuracy (speed): 0.201720
eval mean loss: 0.605269
eval accuracy (angle): 0.094415
eval accuracy (speed): 0.259309
Starting epoch 2
mean loss: 0.748471
accuracy (angle): 0.132937
accuracy (speed): 0.200728
eval mean loss: 0.407079
eval accuracy (angle): 0.162234
eval accuracy (speed): 0.340426
Starting epoch 3
mean loss: 0.521057
accuracy (angle): 0.168320
accuracy (speed): 0.287368
eval mean loss: 0.355005
eval accuracy (angle): 0.190160
eval accuracy (speed): 0.344415
Starting epoch 4
mean loss: 0.448503
accuracy (angle): 0.185185
accuracy (speed): 0.309193
eval mean loss: 0.245980
eval accuracy (angle): 0.220745
eval accuracy (speed): 0.414894
Starting epoch 5
mean loss: 0.421437
accuracy (angle): 0.200066
accuracy (speed): 0.303571
eval

eval mean loss: 0.137281
eval accuracy (angle): 0.385638
eval accuracy (speed): 0.630319
Starting epoch 46
mean loss: 0.193730
accuracy (angle): 0.371362
accuracy (speed): 0.561508
eval mean loss: 0.120249
eval accuracy (angle): 0.406915
eval accuracy (speed): 0.655585
Starting epoch 47
mean loss: 0.172655
accuracy (angle): 0.392857
accuracy (speed): 0.581349
eval mean loss: 0.091484
eval accuracy (angle): 0.412234
eval accuracy (speed): 0.670213
Starting epoch 48
mean loss: 0.158142
accuracy (angle): 0.388558
accuracy (speed): 0.610119
eval mean loss: 0.084426
eval accuracy (angle): 0.433511
eval accuracy (speed): 0.690160
Starting epoch 49
mean loss: 0.166168
accuracy (angle): 0.388558
accuracy (speed): 0.616071
eval mean loss: 0.114580
eval accuracy (angle): 0.396277
eval accuracy (speed): 0.676862
Starting epoch 50
mean loss: 0.152609
accuracy (angle): 0.399471
accuracy (speed): 0.625000
eval mean loss: 0.150245
eval accuracy (angle): 0.406915
eval accuracy (speed): 0.675532
Starti

mean loss: 0.130299
accuracy (angle): 0.470899
accuracy (speed): 0.686839
eval mean loss: 0.116812
eval accuracy (angle): 0.417553
eval accuracy (speed): 0.680851
Starting epoch 92
mean loss: 0.121397
accuracy (angle): 0.474206
accuracy (speed): 0.699735
eval mean loss: 0.189692
eval accuracy (angle): 0.365691
eval accuracy (speed): 0.627660
Starting epoch 93
mean loss: 0.144449
accuracy (angle): 0.453704
accuracy (speed): 0.668981
eval mean loss: 0.134875
eval accuracy (angle): 0.417553
eval accuracy (speed): 0.684840
Starting epoch 94
mean loss: 0.150652
accuracy (angle): 0.487765
accuracy (speed): 0.696429
eval mean loss: 0.125407
eval accuracy (angle): 0.414894
eval accuracy (speed): 0.668883
Starting epoch 95
mean loss: 0.133683
accuracy (angle): 0.479167
accuracy (speed): 0.680556
eval mean loss: 0.092597
eval accuracy (angle): 0.461436
eval accuracy (speed): 0.738032
Starting epoch 96
mean loss: 0.152701
accuracy (angle): 0.436177
accuracy (speed): 0.647817
eval mean loss: 0.096